In [1]:
  import tensorflow as tf

In [2]:
train_dir = "/kaggle/input/dataset/dataset/train"
test_dir = "/kaggle/input/dataset/dataset/test"

In [3]:
IMG_SIZE = (124, 124)
BATCH_SIZE = 32

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

Found 6675 files belonging to 8 classes.


In [5]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 1794 files belonging to 8 classes.


In [6]:
class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)

Classes: ['Textile_waste', 'construction_demolition_waste', 'hazardous_waste', 'kitchen_waste', 'medical', 'organic_waste', 'recycled_waste', 'sanitary_waste']


In [11]:
import os
from PIL import Image

dataset_path = train_dir  # Update with your dataset path

for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            try:
                with Image.open(file_path) as img:
                    img_format = img.format
                    print(f"File: {file} | Format: {img_format}")  # Print format of all images
                    
                    if img_format.lower() != "png":
                        print(f"❌ Non-PNG Image: {file_path} | Detected Format: {img_format}")
            except Exception as e:
                print(f"⚠️ Error reading {file_path}: {e}")  # Handle corrupt files


File: tetra_pak 76.png | Format: PNG
File: beverage_cans 35.png | Format: PNG
File: construction_scrap 67.png | Format: PNG
File: paper 501.png | Format: PNG
File: paper_cups 64.png | Format: PNG
File: construction_scrap 23.png | Format: PNG
File: plastic_bottle 23.png | Format: PNG
File: glass 66.png | Format: PNG
File: plastic_bag 115.png | Format: PNG
File: cardboard 76.png | Format: PNG
File: beverage_cans 11.png | Format: PNG
File: metal_objects 23.png | Format: PNG
File: glass 32.png | Format: PNG
File: beverage_cans 125.png | Format: PNG
File: construction_scrap 14.png | Format: PNG
File: paper 56.png | Format: PNG
File: beverage_cans 63.png | Format: PNG
File: spray_cans 47.png | Format: PNG
File: tetra_pak 12.png | Format: PNG
File: cardboard 22.png | Format: PNG
File: glass 22.png | Format: PNG
File: plastic_bottle 98.png | Format: PNG
File: paper 14.png | Format: PNG
File: plastic_bag 134.png | Format: PNG
File: construction_scrap 58.png | Format: PNG
File: plastic_bottle 10

In [8]:
def split_train_validation(dataset, validation_split=0.1):
    dataset_size = tf.data.experimental.cardinality(dataset).numpy()
    val_size = int(validation_split * dataset_size)

    val_ds = dataset.take(val_size)
    train_ds = dataset.skip(val_size)

    return train_ds, val_ds

In [11]:
train_ds, val_ds = split_train_validation(train_ds)

In [12]:
print(f"Train: {train_ds.cardinality().numpy()}, Validation: {val_ds.cardinality().numpy()}, Test: {test_ds.cardinality().numpy()}")

Train: 171, Validation: 18, Test: 57


In [15]:
from tensorflow.keras.layers import Rescaling, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras import Sequential

preprocessing_layer = Sequential([
    Rescaling(1./255),
    RandomFlip("horizontal"),
    RandomRotation(0.2),
    RandomZoom(0.2)
])

In [16]:
train_ds = train_ds.map(lambda x, y: (preprocessing_layer(x), y))
val_ds = val_ds.map(lambda x, y: (preprocessing_layer(x), y))
test_ds = test_ds.map(lambda x, y: (preprocessing_layer(x), y))

In [17]:
from tensorflow.keras.layers import Input, SeparableConv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense, Dropout, Add
from tensorflow.keras.models import Model

def build_dp_cnn(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # === Parallel Convolutions ===
    conv1 = SeparableConv2D(128, (11, 11), padding="same")(inputs)
    conv2 = SeparableConv2D(128, (9, 9), padding="same")(inputs)
    conv3 = SeparableConv2D(128, (7, 7), padding="same")(inputs)
    conv4 = SeparableConv2D(128, (5, 5), padding="same")(inputs)
    conv5 = SeparableConv2D(128, (3, 3), padding="same")(inputs)

    # Batch Norm & Activation
    act1 = Activation("relu")(BatchNormalization()(conv1))
    act2 = Activation("relu")(BatchNormalization()(conv2))
    act3 = Activation("relu")(BatchNormalization()(conv3))
    act4 = Activation("relu")(BatchNormalization()(conv4))
    act5 = Activation("relu")(BatchNormalization()(conv5))

    # Merge parallel layers
    merged = Add()([act1, act2, act3, act4, act5])

    # === Depthwise Separable Convolutions ===
    conv6 = SeparableConv2D(128, (3, 3), activation="relu", padding="same")(merged)
    conv6 = BatchNormalization()(conv6)
    conv6 = MaxPooling2D(pool_size=(2, 2))(conv6)

    conv7 = SeparableConv2D(64, (3, 3), activation="relu", padding="same")(conv6)
    conv7 = BatchNormalization()(conv7)
    conv7 = MaxPooling2D(pool_size=(2, 2))(conv7)

    conv8 = SeparableConv2D(32, (3, 3), activation="relu", padding="same")(conv7)
    conv8 = BatchNormalization()(conv8)
    conv8 = MaxPooling2D(pool_size=(2, 2))(conv8)

    conv9 = SeparableConv2D(16, (3, 3), activation="relu", padding="same")(conv8)
    conv9 = BatchNormalization()(conv9)
    conv9 = MaxPooling2D(pool_size=(2, 2))(conv9)

    # === Fully Connected Layers ===
    flatten = Flatten()(conv9)
    dense1 = Dense(1024, activation="relu")(flatten)
    dense1 = Dropout(0.5)(dense1)

    dense2 = Dense(512, activation="relu")(dense1)
    dense2 = Dropout(0.5)(dense2)

    outputs = Dense(num_classes, activation="softmax")(dense2)

    model = Model(inputs, outputs)
    return model




In [18]:
# Build Model
model = build_dp_cnn((124, 124, 3), num_classes)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 124, 124, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ separable_conv2d          │ (None, 124, 124, 128)  │            875 │ input_layer_1[0][0]    │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ separable_conv2d_1        │ (None, 124, 124, 128)  │            755 │ input_layer_1[0][0]    │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ separable_conv2d_2        │ (None, 124, 124, 128)  │            659 │ input_layer_1[0][0]    │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ separable_conv2d_3        │ (None, 124, 124, 128)  │            587 │ input_layer_1[0][0]    │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ separable_conv2d_4        │ (None, 124, 124, 128)  │            539 │ input_layer_1[0][0]    │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 124, 124, 128)  │            512 │ separable_conv2d[0][0] │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 124, 124, 128)  │            512 │ separable_conv2d_1[0]… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 124, 124, 128)  │            512 │ separable_conv2d_2[0]… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 124, 124, 128)  │            512 │ separable_conv2d_3[0]… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 124, 124, 128)  │            512 │ separable_conv2d_4[0]… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 124, 124, 128)  │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 124, 124, 128)  │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 124, 124, 128)  │              0 │ batch_normalization_2… │
├──────────────────────

 Total params: 1,370,223 (5.23 MB)

 Trainable params: 1,368,463 (5.22 MB)

 Non-trainable params: 1,760 (6.88 KB)

In [19]:
from tensorflow.keras.optimizers import Adam

In [20]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [21]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=150,
    batch_size=32
)

Epoch 1/150
171/171 ━━━━━━━━━━━━━━━━━━━━ 90s 377ms/step - accuracy: 0.2490 - loss: 2.4053 - val_accuracy: 0.1059 - val_loss: 3.0029
Epoch 2/150
171/171 ━━━━━━━━━━━━━━━━━━━━ 53s 270ms/step - accuracy: 0.3815 - loss: 1.6792 - val_accuracy: 0.1597 - val_loss: 3.5701
Epoch 3/150
171/171 ━━━━━━━━━━━━━━━━━━━━ 55s 287ms/step - accuracy: 0.4264 - loss: 1.5497 - val_accuracy: 0.1788 - val_loss: 3.1358
Epoch 4/150
171/171 ━━━━━━━━━━━━━━━━━━━━ 55s 283ms/step - accuracy: 0.4390 - loss: 1.4532 - val_accuracy: 0.4358 - val_loss: 1.4793
Epoch 5/150
171/171 ━━━━━━━━━━━━━━━━━━━━ 57s 288ms/step - accuracy: 0.4597 - loss: 1.4369 - val_accuracy: 0.4705 - val_loss: 1.4590
Epoch 6/150
171/171 ━━━━━━━━━━━━━━━━━━━━ 52s 265ms/step - accuracy: 0.4758 - loss: 1.3700 - val_accuracy: 0.5469 - val_loss: 1.2327
Epoch 7/150
171/171 ━━━━━━━━━━━━━━━━━━━━ 52s 272ms/step - accuracy: 0.5052 - loss: 1.3206 - val_accuracy: 0.4809 - val_loss: 1.4323
Epoch 8/150
171/171 ━━━━━━━━━━━━━━━━━━━━ 55s 280ms/step - accuracy: 0.5024 -

In [23]:
model.save("/kaggle/working/waste_classification_model_dp_cnn.keras")

In [27]:
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

57/57 ━━━━━━━━━━━━━━━━━━━━ 12s 203ms/step - accuracy: 0.7355 - loss: 0.9128
Test Accuracy: 66.11%
